<a href="https://colab.research.google.com/github/bnsreenu/python_for_microscopists/blob/master/339_surrogate_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://youtu.be/6GxFAw2IFyw

Surrogate optimization is a method used to solve optimization problems that are expensive or time-consuming to evaluate directly. It relies on constructing a surrogate model (also known as a metamodel) that approximates the objective function based on a limited number of evaluations. The surrogate model is then used to guide the search for the optimal solution. This approach is particularly useful when dealing with complex simulations, physical experiments, or other computationally expensive tasks.
<p>
Unlike traditional metaheuristic approaches like Particle Swarm Optimization (PSO) and Simulated Annealing (SA), surrogate optimization is not strictly a metaheuristic. It can be combined with metaheuristics or other optimization techniques to enhance their efficiency.
<p>
Surrogate optimization is often considered a Bayesian approach because it incorporates Bayesian principles in its methodology, particularly through the use of Gaussian Processes (GPs) and Bayesian optimization techniques.
<p>
To get a practical understanding, let's implement a simple example of surrogate optimization using the same objective function from our PSO tutorial. We'll use a popular surrogate model called Gaussian Process (GP) and the Bayesian Optimization framework.

In [ ]:
import numpy as np
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize

# Define the objective function (Simple function with a solution of, [4, 5, -6])
def objective_function(params):
    x, y, z = params[0], params[1], params[2]
    return (x-4)**2 + (y-5)**2 + (z+6)**2

# Define the bounds of the search space
bounds = np.array([[-10, -10, -10], [10, 10, 10]])


Randomly sample a few initial points in the search space and evaluate the objective function at these points. We use a Gaussian Process model to approximate the objective function. The Matern kernel is commonly used in surrogate optimization. <p>


In [ ]:
# Randomly sample initial points
n_initial_points = 5
initial_points = np.random.uniform(low=bounds[0], high=bounds[1], size=(n_initial_points, 3))
initial_values = np.array([objective_function(p) for p in initial_points])
print("Initial points are: ", initial_points)
print("Initial values for the objective function are: ", initial_values)


Initial points are:  [[-4.68611508 -5.81172591  4.38565136]
 [ 3.69743208  0.93772233  7.47319329]
 [-0.65339392 -4.81721076 -1.74069166]
 [ 0.75507018 -8.10809832  8.81978001]
 [-8.68497866 -9.24062617 -6.79973467]]
Initial values for the objective function are:  [300.20376651 198.12058467 136.17340963 401.97769056 364.34369271]


A Gaussian Process (GP) is a powerful statistical model used for regression tasks. It is particularly well-suited for surrogate modeling in optimization because it provides a probabilistic prediction, which includes both a mean prediction and an uncertainty estimate. This property is essential for making decisions about where to sample next in the search space. In summary, it helps estimate the objective function at unobserved points. <p>
The Matern kernel is a popular choice for the covariance function (or kernel) in Gaussian Processes. It is more flexible than the Radial Basis Function (RBF) kernel because it has an additional parameter, nu, which controls the smoothness of the function. The parameter nu affects the smoothness of the predicted function: smaller values of nu lead to rougher functions, while larger values lead to smoother functions.
<br> nu=1.5 or nu=2.5: Commonly used values that provide a good balance between smoothness and flexibility.

In [ ]:
# Train the initial Gaussian Process model
kernel = Matern(nu=2.5)  #Create a Matern kernel

#Initialize the GP model with the specified Matern kernel and other parameters.
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=1e-6)
gp.fit(initial_points, initial_values) #Train the GP model on the initial set of points and their corresponding objective function values.

/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


GaussianProcessRegressor(alpha=1e-06, kernel=Matern(length_scale=1, nu=2.5),
                         n_restarts_optimizer=10)

The acquisition function guides the search for the next point to evaluate. We use the Expected Improvement (EI) criterion, which balances exploration and exploitation. <p>
The acquisition function's role is to guide the search for the optimal solution by determining which point in the search space should be evaluated next. The acquisition function balances exploration (searching new areas) and exploitation (refining known good areas).

In [ ]:

# Define the acquisition function (Expected Improvement)
# Note that norm.cdf(Z) is the cumulative distribution function of the standard normal distribution,
# representing the probability that the improvement will be achieved.
# norm.pdf(Z) is the probability density function of the standard normal distribution,
# accounting for the uncertainty in the prediction.
# More information? https://ekamperi.github.io/machine%20learning/2021/06/11/acquisition-functions.html
def acquisition_function(x, gp, y_min):
    mu, sigma = gp.predict(x.reshape(1, -1), return_std=True)  # Predicted Mean (mu) and Standard Deviation (sigma)
    with np.errstate(divide='warn'):    #  warning will be issued whenever a division by zero occurs during the execution of the code block
        imp = y_min - mu  #improvement: If imp is positive, it indicates that the point x has the potential to improve upon the best known value.
        Z = imp / sigma  # Standardized improvement, normalized by the standard deviation to account for the uncertainty in the prediction
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z) # Expected Improvement, combine the improvement and the probability of achieving that improvement.
        ei[sigma == 0.0] = 0.0  # If sigma is zero, meaning the prediction is certain, the expected improvement is set to zero to avoid division by zero.
    return -ei

# Perform the optimization:
# First, initialize the Gaussian Process (GP) model with some initial points and their corresponding objective function values.
# Then, optimize by iterating for a specified number of iterations to perform:
    # Candidate selection
    # Evaluate the objective function
    # Update the GP model
n_iterations = 50
for i in range(n_iterations):
    # Find the next point to evaluate using multiple starting points for robustness
    candidates = np.random.uniform(low=bounds[0], high=bounds[1], size=(10, 3))  # 10 random candidates

    # optimization of the acquisition function for each candidate point in the search space
    # and store the optimization results (minimum point and function value)  - in the res_list.
    # To find the minimum of the objective function, let us use L-BGFS-B
    # 'L-BFGS-B' refers to Limited-memory Broyden–Fletcher–Goldfarb–Shanno algorithm
    # with box constraints, which is suitable for optimization problems with bounds
    res_list = [minimize(lambda x: acquisition_function(x, gp, initial_values.min()),
                         x0=candidate, bounds=bounds.T, method='L-BFGS-B') for candidate in candidates]
    res = min(res_list, key=lambda x: x.fun)
    next_point = res.x

    # Evaluate the objective function at the new point
    next_value = objective_function(next_point)

    # Update the Gaussian Process model with the new point
    initial_points = np.vstack((initial_points, next_point))
    initial_values = np.append(initial_values, next_value)
    gp.fit(initial_points, initial_values)

    # Print the progress
    print(f'Iteration {i+1}: Best Cost = {initial_values.min():.6f}')

# Print the results
best_index = np.argmin(initial_values)
global_best_position = initial_points[best_index]
global_best_cost = initial_values[best_index]

print('Global Best Position:', global_best_position)
print('Global Best Cost:', global_best_cost)


Iteration 1: Best Cost = 77.000000
Iteration 2: Best Cost = 77.000000
Iteration 3: Best Cost = 77.000000
Iteration 4: Best Cost = 77.000000
Iteration 5: Best Cost = 77.000000
Iteration 6: Best Cost = 17.765558
Iteration 7: Best Cost = 11.313780
Iteration 8: Best Cost = 11.313780
Iteration 9: Best Cost = 11.313780
Iteration 10: Best Cost = 11.313780
Iteration 11: Best Cost = 0.011491
Iteration 12: Best Cost = 0.011491
Iteration 13: Best Cost = 0.011491
Iteration 14: Best Cost = 0.011491
Iteration 15: Best Cost = 0.011491
Iteration 16: Best Cost = 0.011491
Iteration 17: Best Cost = 0.011491
Iteration 18: Best Cost = 0.011491
Iteration 19: Best Cost = 0.011491
Iteration 20: Best Cost = 0.011491
Iteration 21: Best Cost = 0.011491
Iteration 22: Best Cost = 0.011491
Iteration 23: Best Cost = 0.011491
Iteration 24: Best Cost = 0.011491
Iteration 25: Best Cost = 0.011491
Iteration 26: Best Cost = 0.011491
Iteration 27: Best Cost = 0.011491
Iteration 28: Best Cost = 0.011491
Iteration 29: Best 

Now that you understand the inner workings of the surrogate optimization, let us use a library that offers this optimization out of the box.

In [ ]:
!pip install scikit-optimize

In [ ]:
from skopt import gp_minimize
from skopt.learning import GaussianProcessRegressor
from skopt.learning.gaussian_process.kernels import Matern
from skopt.acquisition import gaussian_ei
import numpy as np

# Define the objective function
def objective_function(params):
    x, y, z = params[0], params[1], params[2]
    return (x-4)**2 + (y-5)**2 + (z+6)**2

# Define the bounds of the search space
bounds = [(-10, 10), (-10, 10), (-10, 10)]

# Define the initial points and values
initial_points = []
initial_values = []

# Define the GP model
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6)

# Perform surrogate optimization using gp_minimize
res = gp_minimize(objective_function, bounds, acq_func="EI", n_calls=30, n_random_starts=10, random_state=42)

# Print the results
print('Global Best Position:', res.x)
print('Global Best Cost:', res.fun)


/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [4, 5, -6] before, using random point [-4, 6, 7]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [4, 5, -6] before, using random point [-5, -9, -4]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [4, 5, -6] before, using random point [-4, -5, -9]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [4, 5, -6] before, using random point [10, -7, -8]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [4, 5, -6] before, using random point [0, -7, -10]
  warnings.warn(
/usr/local/lib/py

Global Best Position: [4, 5, -6]
Global Best Cost: 0


Let's consider a more complex objective function for demonstration purposes. We'll use the Rosenbrock function, which is a commonly used benchmark function in optimization. The Rosenbrock function is defined as: <p>

f(x,y)=(a−x)<sup>2</sup> + b(y−x<sup>2</sup>)<sup>2</sup>
<p>
where a and b are constants, typically a=1 and b=100. <p>

It has a global minimum at x=a and y=a<sup>2</sup> <p>
<p> Let us define bounds to be (-5, 5), (-5, 5) in which case a spossible solution would be x=1 and y=1. <p>
Then, change the bounds to (2, 5), (-5, 5). You should see the solution to be x=2 and y=4.

In [ ]:
from skopt import gp_minimize
from skopt.learning import GaussianProcessRegressor
from skopt.learning.gaussian_process.kernels import Matern
import numpy as np

# Define the Rosenbrock function
def rosenbrock(x):
    a = 1
    b = 100
    return (a - x[0])**2 + b * (x[1] - x[0]**2)**2

# Define the bounds of the search space
bounds = [(2, 5), (-5, 5)]

# Define the GP model
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6)

# Perform surrogate optimization using gp_minimize
res = gp_minimize(rosenbrock, bounds, n_calls=30, n_random_starts=10, random_state=42)

# Print the results
print('Global Best Position:', res.x)
print('Global Best Cost:', res.fun)


/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [2, 5] before, using random point [3, -1]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [2, 5] before, using random point [4, 1]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [2, 5] before, using random point [4, 3]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [2, 5] before, using random point [3, -1]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [2, 5] before, using random point [4, -4]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimi

Global Best Position: [2, 4]
Global Best Cost: 1
